### ***9***

# MAP FOR MULTICLASS CLASSIFICATION

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%reset-f

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy
path = '/content/drive/MyDrive/ML/data_q6_q7.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
n = len(insts[0,:])-1 #no of features  
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std() #Normalize the data
np.random.shuffle(insts)

In [5]:
def cal_mean(insts,m,n): 
  mean = np.ones(n)
  for i in range(n):
    mean[i] = np.sum(insts[:,i])/m
  return mean

def cal_prior(insts,c): #insts of training set , no of classes
  p= np.ones(c)
  l = len(insts[:,-1])
  for i in range(c):
    p[i] = np.count_nonzero(insts[:,-1] == i+1)/l
  return p # returns prior probabilities matrix

In [6]:
K = 5 # no of folds
overall_accuracy = 0
accuracy_class1 = 0
accuracy_class2 = 0
accuracy_class3 = 0
for k in range(K):
  insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  P_y = cal_prior(insts_tr,3) #prior probability vector
  
  insts_y1 = insts_tr[insts_tr[:,-1]==1] #insts for class 1 in training set
  insts_y2 = insts_tr[insts_tr[:,-1]==2] #insts for class 2 in training set
  insts_y3 = insts_tr[insts_tr[:,-1]==3] #insts for class 3 in training set
  mean1 = cal_mean(insts_y1,len(insts_y1[:,0]),n) #mean vector for class 1
  mean2 = cal_mean(insts_y2,len(insts_y2[:,0]),n) #mean vector for class 2
  mean3 = cal_mean(insts_y3,len(insts_y3[:,0]),n) #mean vector for class 3
  cov1  = np.cov((insts_y1[:,0:n].astype(np.float32)).T)
  cov2  = np.cov((insts_y2[:,0:n].astype(np.float32)).T)
  cov3  = np.cov((insts_y3[:,0:n].astype(np.float32)).T)
  coeff1 = 1/(((2*3.14)**(n/2))*np.linalg.det(cov1)**0.5)
  coeff2 = 1/(((2*3.14)**(n/2))*np.linalg.det(cov2)**0.5)
  coeff3 = 1/(((2*3.14)**(n/2))*np.linalg.det(cov3)**0.5)
  y_pred = []
  y_acu = insts_te[:,-1]
  l = np.ones(3)
  for i in range(m_te):
    l[0] = coeff1*np.exp(-0.5*np.dot(np.dot((insts_te[i,0:n] - mean1),np.linalg.inv(cov1)),(insts_te[i,0:n] - mean1).T))
    l[1] = coeff2*np.exp(-0.5*np.dot(np.dot((insts_te[i,0:n] - mean2),np.linalg.inv(cov2)),(insts_te[i,0:n] - mean2).T))
    l[2] = coeff3*np.exp(-0.5*np.dot(np.dot((insts_te[i,0:n] - mean3),np.linalg.inv(cov3)),(insts_te[i,0:n] - mean3).T))
    MAP = np.ones(3) # a postirior probability 
    e = l[0]*P_y[0] + l[1]*P_y[1] + l[2]*P_y[2] #evidence
    for i in range(3):
      MAP[i] = (l[i]*P_y[i])/e
    y_pred.append(1+ np.argmax(MAP))
  y_actual = pd.Series(y_acu, name='Actual')
  y_pred = pd.Series(y_pred, name='Predicted')
  confmat = pd.crosstab(y_actual, y_pred)
  confmat = np.asarray(confmat)
  overall_accuracy += (confmat[0][0] + confmat[1][1] + confmat[2][2])/sum(sum(confmat))
  accuracy_class1 += confmat[0][0]/sum(confmat[0])
  accuracy_class2 += confmat[1][1]/sum(confmat[1])
  accuracy_class3 += confmat[2][2]/sum(confmat[2])
print('Overall Accuracy:', overall_accuracy/K ,'\nClass1 Accuracy:',accuracy_class1/K,'\nClass2 Accuracy:',accuracy_class2/K,'\nClass3 Accuracy:',accuracy_class3/K ) 


Overall Accuracy: 0.9475029036004645 
Class1 Accuracy: 0.8946212121212123 
Class2 Accuracy: 0.9894736842105264 
Class3 Accuracy: 0.9566666666666667
